In [3]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/home/srivathsa/projects/voxelmorph')

import os
from voxelmorph.tf.airnet import DenseNetEncoder
from voxelmorph.tf.networks import AIRNet
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (10, 8)

# from tensorflow.keras.layers import Input, Concatenate, Dense
# from tensorflow.keras.models import Model

from keras.layers import Input, Concatenate, Dense
from keras.models import Model

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

<Figure size 720x576 with 0 Axes>

In [ ]:
img1 = Input(shape=(128, 256, 256, 1))
img2 = Input(shape=(128, 256, 256, 1))
x1, x2 = DenseNetEncoder(img1, img2, depth=7, nb_dense_block=6, growth_rate=8, nb_filter=16)
ftr_cat = Concatenate()([x1, x2])
d1 = Dense(1024)(ftr_cat)
d2 = Dense(512)(d1)
d3 = Dense(128)(d2)
d4 = Dense(64)(d3)
affn_out = Dense(12)(d4)

airnet = Model(inputs=[img1, img2], outputs=[affn_out], name="AIRNet")
airnet.summary(line_length=120)

In [4]:
airnet = AIRNet(inshape=(128, 256, 256))
airnet.model.load_weights('/home/srivathsa/projects/studies/gad/vmorph/runs/20211006_212811-brats/ckps/0150.h5')

Model: "model_1"
______________________________________________________________________________________________________________
Layer (type)                        Output Shape            Param #      Connected to                         
airnet_source_input (InputLayer)    (None, 128, 256, 256, 1 0                                                 
______________________________________________________________________________________________________________
airnet_target_input (InputLayer)    (None, 128, 256, 256, 1 0                                                 
______________________________________________________________________________________________________________
pre_conv (Conv3D)                   (None, 128, 256, 256, 1 144          airnet_source_input[0][0]            
                                                                         airnet_target_input[0][0]            
_______________________________________________________________________________________________

In [ ]:
aff_param = airnet.model.get_layer('airnet_aff_mtx')
tmp_model = Model(inputs=[airnet.model.layers[0].input, airnet.model.layers[1].input], outputs=aff_param.output)
tmp_model.load_weights('/home/srivathsa/projects/studies/gad/vmorph/runs/20211006_212811-brats/ckps/0080.h5', by_name=True)

In [ ]:
ip = np.load('/home/srivathsa/projects/studies/gad/vmorph/data/brats/val/BraTS2021_00006.npz')['data']
ip1 = ip[0][None, ..., None]
ip2 = ip[1][None, ..., None]

aff_mtx = tmp_model.predict([ip2, ip1])
mov_img = airnet.model.predict([ip2, ip1])

In [ ]:
aff_mtx

In [ ]:
img1 = ip[0, 64]
img2 = ip[1, 64]
img3 = mov_img[0, 64, ..., 0]
plt.imshow(np.hstack([img1, img3 - img2]))

In [ ]:
airnet = AIRNet(inshape=(128, 256, 256), num_filters=64, growth_rate=16, enc_depth=10)